### 1.读取和获取数据集

In [57]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sys
import d2lzh_pytorch as d2l
sys.path.append("..")

print(torch.__version__)
torch.set_default_tensor_type(torch.FloatTensor)

ModuleNotFoundError: No module named 'torchtext'

训练数据集包括1460个样本,80个特征和1个标签  
测试数据集包括1459个样本和80个特征

In [13]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')
print(train_data.shape)
print(test_data.shape) 

(1460, 81)
(1459, 80)


In [14]:
# 查看前4个特征,后两个特征和标签(SalePrice))
train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [20]:
all_features = pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))
all_features.shape

(2919, 79)

### 2.预处理数据

#### 对连续数值的特征做标准化　　
* 将该特征的每个值先减去均值，再除以标准差．　　　　
* 对于缺失的特征值，将其替换成该特征的均值．　　

In [23]:
# dataframe中的字符串dtype=object

# pandas.DataFrame.dtypes：pandas提供的这个方法用于返回这个DataFrame中所有列的dtype，以一个pandas.Series形式返回。这个Series的index就是DataFrame的各列名。

# numeric_features接受的是数据全部为数字的列名.  
# 含NaN的列也为object,(此处有疑问???)
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
print(numeric_features)

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')


In [30]:
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / x.std()
)
# 标准化后，均值为０，故直接用０来替换缺失值
# (这两个步骤是否考虑换一下???)
# fillna(number) 直接对数据块中为NaN替换为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

#### 将离散数值转成指示特征
* 相当于一个onehot操作

In [32]:
# dummy_na=True将缺失值也当作合法的特征值并为其创建指示特征
# 特征数从79增加到了331
all_features = pd.get_dummies(all_features,dummy_na=True)
all_features.shape

(2919, 331)

* 通过values属性得到numpy格式的数据,并转成Tensor方便后面的训练.

In [40]:
# train_data.shape=(1460, 81)
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float)
test_features = torch.tensor(all_features[n_train:-1].values, dtype=torch.float)
train_labels = torch.tensor(train_data.SalePrice.values, dtype=torch.float)

### 3.训练模型

* 使用一个基本的线性回归模型和平方损失函数来训练模型

In [41]:
loss = torch.nn.MSELoss()

def get_net(feature_num):
    net = nn.Linear(feature_num, 1)
    for param in net.parameters():
        nn.init.normal_(param, mean=0, std=0.01)
    return net

* 定义从来凭借模型的对数均方根误差

``还没看懂?????``  
net(features)返回的应该是一个张量,这个张量具体是什么东西.

In [42]:
# 把features传入到net中,对其输出与labels计算误差
def log_mrse(net, features, labels):
    with torch.no_grad():
        # 将小于1的值设成1,使得取对数时数值更加稳定
        clipped_preds = torch.max(net(features), torch.tensor(1.0))
        rmse = torch.sqrt(loss(clipped_preds.log(), labels.log()))
    return rmse.item()

``net = net.float()``什么意思?????  
``net(X.float())``什么意思??????????,前面并没有对net进行定义.

In [45]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    # 训练误差和测试误差
    train_ls, test_ls = [], []
    dataset = torch.utils.data.TensorDataset(train_data, train_labels)
    train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
    # 这里使用了Adam优化算法
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=weight_decay)
    net = net.float()
    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X.float()), y.float)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_ls.append(log_mrse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_mrse(net, test_features, test_labels))
    return train_ls, test_ls





### 4.K折交叉验证

* 得到第i折交叉验证时的训练和验证数据

In [46]:
def get_k_fold_data(k, i, X, y):
    # 返回第i折交叉验证时所需要的训练和验证数据
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j+1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train =  X_part, y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
    return X_train, y_train, X_valid, y_valid


* 在K折交叉验证中我们训练K次并返回训练和验证的平均误差

``d21``需要添加,待解决????????

In [54]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net(X_train.shape[1])
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse',
            range(1, num_epochs + 1), valid_ls, ['train', 'valid'])
        print('fold %d, train rmse %f, valid rmse %f' % (i, train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k


### 5.模型选择

In [55]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
print('%d-fold validation:avg train rmse %f, avg valid rmse %f' % (k, train_l, valid_l))


TypeError: 'numpy.int64' object is not callable

### 6.预测并提交结果

``还未搞懂?????????``